In [1]:
import plotly.express as px
import plotly.io as pio
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import os

In [ ]:
# レンダラーの指定 (ノートブック上で動作させる場合)
pio.renderers.default = "notebook_connected"  # 他に "iframe_connected" なども選べる

# データ読み込み
metadata_path = "../data/processed/paper_metadata.csv"
df = pd.read_csv(metadata_path)

In [3]:
# ターム抽出関数
def split_terms(cell):
    return [t.strip() for t in str(cell).split(',') if t.strip()]

# Data Types と Methods のタームセット作成
data_terms = [t for cell in df['data_types'].dropna() for t in split_terms(cell)]
unique_data_terms = list(dict.fromkeys(data_terms))
method_terms = [t for cell in df['methods'].dropna() for t in split_terms(cell)]
unique_method_terms = list(dict.fromkeys(method_terms))

# ベクトル化 (TF-IDF)
vectorizer_data = TfidfVectorizer()
X_data = vectorizer_data.fit_transform(unique_data_terms).toarray()
vectorizer_method = TfidfVectorizer()
X_method = vectorizer_method.fit_transform(unique_method_terms).toarray()

# t-SNE 次元削減
perp_data = min(30, len(unique_data_terms)-1)
perp_method = min(30, len(unique_method_terms)-1)
coords_data = TSNE(n_components=2, random_state=42, init='pca', perplexity=perp_data).fit_transform(X_data)
coords_method = TSNE(n_components=2, random_state=42, init='pca', perplexity=perp_method).fit_transform(X_method)

# KMeans クラスタリング (15 クラスタ)
n_clusters = 20
labels_data = KMeans(n_clusters=n_clusters, random_state=42).fit_predict(coords_data)
labels_method = KMeans(n_clusters=n_clusters, random_state=42).fit_predict(coords_method)

In [4]:
# DataFrame 作成
df_plot_data = pd.DataFrame({
    'term': unique_data_terms,
    'x': coords_data[:, 0],
    'y': coords_data[:, 1],
    'cluster': labels_data.astype(str)
})
df_plot_method = pd.DataFrame({
    'term': unique_method_terms,
    'x': coords_method[:, 0],
    'y': coords_method[:, 1],
    'cluster': labels_method.astype(str)
})

In [5]:
from langchain_ollama import OllamaLLM
import pandas as pd
import time

llm = OllamaLLM(model="llama3")

def summarize_clusters(df, term_col="term", cluster_col="cluster", max_terms=10, c=""):
    summary_map = {}
    for cl in sorted(df[cluster_col].unique()):
        terms = df.loc[df[cluster_col] == cl, term_col].tolist()
        # 1) 頻度や出現順で上位 max_terms 件に絞る
        terms_for_prompt = terms[:max_terms]
        if c == "data_type":
            prompt = (
                "以下の用語から、このクラスタのテーマを20字以内の日本語のラベルで要約してください。ラベルの末尾は「データ」にしてください。既に「データ」が付いている場合は不要です。ラベルには「」をつけないでください。"
                "ラベル以外は何も書かないでください。「ラベル：」や「データ：」といった言葉も不要です。\n\n"
                f"用語一覧: {', '.join(terms_for_prompt)}\n"
                "ラベル:"
            )
        elif c == "method":
            prompt = (
                "以下の用語から、このクラスタのテーマを20字以内の日本語のラベルで要約してください。ラベルには「」をつけないでください。"
                "ラベル以外は何も書かないでください。「ラベル：」といった言葉も不要です。\n\n"
                f"用語一覧: {', '.join(terms_for_prompt)}\n"
                "ラベル:"
            )
        try:
            label = llm(prompt).strip()
            # 応答が明らかにおかしい or 空ならフォールバック
            if not label or len(label) > 30:
                raise ValueError("Invalid label")
        except Exception:
            label = f"cluster_{cl}"
        summary_map[cl] = label
        time.sleep(1)  # 少し休ませる
    df[f"{cluster_col}_label"] = df[cluster_col].map(summary_map)
    return df, summary_map

C:\Users\user\AppData\Local\Temp\ipykernel_13640\3248351655.py:28: LangChainDeprecationWarning:

The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.



In [ ]:
import os
import pandas as pd
import plotly.express as px

df_plot_data, data_labels = summarize_clusters(df_plot_data, "term", "cluster", max_terms=8, c="data_type")
# --- ② Plotlyで散布図作成 ---
fig = px.scatter(
    df_plot_data,
    x="x",
    y="y",
    color="cluster_label", # 要約ラベルで色分け
    hover_name="term",
    # title="Data Types Clustering (with Cluster Labels)",
    width=800, height=600
)

# マーカーの見た目調整（任意）
fig.update_traces(marker=dict(size=10, opacity=0.8))

# 軸と凡例のラベルを整える
fig.update_layout(
    xaxis_title="TSNE 1",
    yaxis_title="TSNE 2",
    legend_title_text="クラスターラベル"
)

# 画像として保存（PNG）
fig.write_image("../data/plots/data_types_clustering.png", format="png", width=800, height=600, scale=2)

# PDF や SVG にも出力可能
fig.write_image("../data/plots/data_types_clustering.pdf", format="pdf", width=800, height=600)
fig.write_image("../data/plots/data_types_clustering.svg", format="svg", width=800, height=600)

# HTMLとして保存
fig.write_html(
    "../data/plots/data_types_clustering.html",
    include_plotlyjs="cdn", # 軽量にする場合
    full_html=True, # フルHTMLドキュメントとして保存
    default_width="100%", # レスポンシブ対応
    default_height="100%"
)

# 描画
fig.show()

In [ ]:
df_plot_method, method_labels = summarize_clusters(df_plot_method, "term", "cluster", max_terms=8, c="method")

fig2 = px.scatter(
    df_plot_method,
    x="x", y="y",
    color="cluster_label",
    hover_name="term",
    # title="Methods Clustering (with Cluster Labels)",
    width=800, height=600
)
fig2.update_traces(marker=dict(size=10, opacity=0.8))
fig2.update_layout(xaxis_title="TSNE 1", yaxis_title="TSNE 2", legend_title_text="クラスターラベル")

# 画像として保存（PNG）
fig2.write_image("../data/plots/methods_clustering.png", format="png", width=800, height=600, scale=2)

# PDF や SVG にも出力可能
fig2.write_image("../data/plots/methods_clustering.pdf", format="pdf", width=800, height=600)
fig2.write_image("../data/plots/methods_clustering.svg", format="svg", width=800, height=600)

# HTMLとして保存
fig2.write_html(
    "../data/plots/methods_clustering.html",
    include_plotlyjs="cdn", # 軽量にする場合
    full_html=True, # フルHTMLドキュメントとして保存
    default_width="100%", # レスポンシブ対応
    default_height="100%"
)

# 描画
fig2.show()